In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

import sys
sys.path.append('/mnt/disk1/yunseob/Pytorch/2_SSM/0_modules')
from SSM_utils import GBdataLoad, B6dataLoad
from SSM_DL_evl import SSM_Model

from sklearn.metrics import roc_curve, auc

# 1. Load Data

In [12]:
data_dir = '/mnt/disk1/yunseob/Pytorch/2_SSM/1_Semblex/05_10p_overlap/SemblexData/wt_4040'

GB_Xs, GB_Ys = GBdataLoad(data_dir, ch = [0, 1], data_type = '2_data')
B6_Xs, B6_Ys = B6dataLoad(data_dir, ch = [0, 1], data_type = '2_data')

In [13]:
len(GB_Xs[0]), len(GB_Xs[1])

(2713, 4240)

In [14]:
[len(i) for i in B6_Xs]

[258, 806, 457, 1202, 734, 783]

In [15]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True, random_state = 44)

for train_idx, test_idx in kf.split(B6_Xs[0]):
    print('test:', test_idx)
    
for train_idx, test_idx in kf.split(GB_Xs[0]):
    print('test:', test_idx)

test: [  3   5   7   9  12  23  28  30  35  36  39  43  44  45  46  47  48  51
  52  54  61  69  73  77  79 102 105 106 107 110 114 115 124 155 157 163
 171 174 175 188 193 198 201 204 210 212 215 228 232 238 244 249]
test: [  6  13  17  19  20  32  37  38  58  60  64  65  68  82  85  88  94 108
 111 125 127 134 135 136 137 138 139 142 145 149 156 158 159 161 176 178
 180 183 185 187 192 196 206 207 209 216 233 236 245 250 252 257]
test: [  0   2  15  24  26  27  33  41  70  71  75  76  80  81  87  89  90  95
  99 112 113 118 126 129 133 147 148 153 154 162 165 167 168 169 170 177
 184 186 190 191 197 200 202 205 208 223 237 242 243 246 247 251]
test: [  4   8  10  11  16  21  22  25  34  40  42  49  53  56  62  63  66  78
  91  97  98 101 104 117 119 121 123 128 130 132 140 141 143 152 160 166
 172 181 182 194 203 211 214 218 220 224 225 229 240 255 256]
test: [  1  14  18  29  31  50  55  57  59  67  72  74  83  84  86  92  93  96
 100 103 109 116 120 122 131 144 146 150 151 164 173 

In [16]:
def CV_test_idx(X, n_splits = 5, shuffle = True, random_state = 44):
    kf = KFold(n_splits, shuffle, random_state)
    test_idx = []
    for _, t_idx in kf.split(X):
        test_idx.append(t_idx)
    return test_idx

G_test_idx = CV_test_idx(GB_Xs[0])
B_test_idx = CV_test_idx(GB_Xs[1])

B1_test_idx = CV_test_idx(B6_Xs[0])
B2_test_idx = CV_test_idx(B6_Xs[1])
B3_test_idx = CV_test_idx(B6_Xs[2])
B4_test_idx = CV_test_idx(B6_Xs[3])
B5_test_idx = CV_test_idx(B6_Xs[4])
B6_test_idx = CV_test_idx(B6_Xs[5])

# 1. Fault detection

In [17]:
model_dir = './model/'
CM_save_dir = './results/1_confusion_mat/'

In [7]:
class_name = ['Good', 'Bad']

In [14]:
GB_test_X.shape

(1391, 2, 40, 40)

In [16]:
GB_test_X[:, 0, :, :].shape

(1391, 40, 40)

In [9]:
GB_10p_A_metrics = []
GB_10p_B_metrics = []
GB_10p_AB_metrics = []

GB_10p_A_cms = []
GB_10p_B_cms = []
GB_10p_AB_cms = []

for i, g_idx, b_idx in zip(range(5), G_test_idx, B_test_idx):

    GB_Xs_cv = [GB_Xs[0][g_idx], GB_Xs[1][b_idx]]
    GB_Ys_cv = [GB_Ys[0][g_idx], GB_Ys[1][b_idx]]

    GB_test_X, GB_test_Y = np.concatenate(GB_Xs_cv), np.concatenate(GB_Ys_cv)

    GB_10p_A_model = SSM_Model(np.expand_dims(GB_test_X[:,0,:,:], axis = 1), GB_test_Y, n_ch = 1, n_cls = 2, class_name = class_name, 
                                model_dir = model_dir, model_name = '{:02d}_GB_10p_A'.format(i+1), batch_size = 100, GPU_idx = 6)
    
    GB_10p_A_model.Evaluation()
    GB_10p_A_metrics.append(GB_10p_A_model.cls_metrics)
    GB_10p_A_cms.append(GB_10p_A_model.cm)
    
    GB_10p_B_model = SSM_Model(np.expand_dims(GB_test_X[:,1,:,:], axis = 1), GB_test_Y, n_ch = 1, n_cls = 2, class_name = class_name, 
                                model_dir = model_dir, model_name = '{:02d}_GB_10p_B'.format(i+1), batch_size = 100, GPU_idx = 6)

    GB_10p_B_model.Evaluation()
    GB_10p_B_metrics.append(GB_10p_B_model.cls_metrics)
    GB_10p_B_cms.append(GB_10p_B_model.cm)
    
    GB_10p_AB_model = SSM_Model(GB_test_X, GB_test_Y, n_ch = 2, n_cls = 2, class_name = class_name, 
                                model_dir = model_dir, model_name = '{:02d}_GB_10p_AB'.format(i+1), batch_size = 100, GPU_idx = 6)
    GB_10p_AB_model.Evaluation()
    GB_10p_AB_metrics.append(GB_10p_AB_model.cls_metrics)
    GB_10p_AB_cms.append(GB_10p_AB_model.cm)

model: 01_GB_10p_A_t_accr_0.9958_t_loss_0.010270_v_accr_0.9892_v_loss_0.052481.pt
epoch: 104, lr (starting at 0.01): 0.01
model: 01_GB_10p_B_t_accr_1.0000_t_loss_0.000000_v_accr_0.9982_v_loss_0.015533.pt
epoch: 151, lr (starting at 0.01): 0.0001
model: 01_GB_10p_AB_t_accr_0.9986_t_loss_0.005841_v_accr_0.9964_v_loss_0.028792.pt
epoch: 113, lr (starting at 0.01): 0.01
model: 02_GB_10p_A_t_accr_1.0000_t_loss_0.000095_v_accr_0.9910_v_loss_0.157374.pt
epoch: 147, lr (starting at 0.01): 0.0001
model: 02_GB_10p_B_t_accr_1.0000_t_loss_0.000000_v_accr_0.9982_v_loss_0.008921.pt
epoch: 154, lr (starting at 0.01): 0.0001
model: 02_GB_10p_AB_t_accr_1.0000_t_loss_0.000346_v_accr_0.9964_v_loss_0.054790.pt
epoch: 109, lr (starting at 0.01): 0.01
model: 03_GB_10p_A_t_accr_1.0000_t_loss_0.000003_v_accr_0.9803_v_loss_0.308828.pt
epoch: 171, lr (starting at 0.01): 0.0001
model: 03_GB_10p_B_t_accr_0.9988_t_loss_0.005019_v_accr_0.9946_v_loss_0.049295.pt
epoch: 77, lr (starting at 0.01): 0.001
model: 03_GB_1

In [17]:
np.asarray(GB_10p_B_cms)

array([[[535,   8],
        [  7, 841]],

       [[531,  12],
        [ 10, 838]],

       [[526,  17],
        [  5, 843]],

       [[535,   7],
        [  3, 845]],

       [[539,   3],
        [ 12, 836]]])

In [18]:
np.save('./results/GB_10p_A_cms', np.asarray(GB_10p_A_cms))
np.save('./results/GB_10p_B_cms', np.asarray(GB_10p_B_cms))
np.save('./results/GB_10p_AB_cms', np.asarray(GB_10p_AB_cms))

In [26]:
GB_10p_AB_metrics

[[0.9842, 0.9836, 0.9906, 0.9871],
 [0.9921, 0.9895, 0.9976, 0.9935],
 [0.9871, 0.9917, 0.987, 0.9893],
 [0.9928, 0.9988, 0.9894, 0.9941],
 [0.995, 0.9988, 0.9929, 0.9958]]

In [25]:
GB_10p_A_metrics

[[0.9784, 0.9904, 0.9741, 0.9822],
 [0.9777, 0.9928, 0.9705, 0.9815],
 [0.9763, 0.9869, 0.9741, 0.9805],
 [0.9698, 0.9786, 0.9717, 0.9751],
 [0.9835, 0.9905, 0.9823, 0.9864]]

In [34]:
avg = np.round(np.mean(100*np.asarray(GB_10p_A_metrics), axis = 0), 2)
avg

array([97.71, 98.78, 97.45, 98.11])

In [35]:
std = np.round(np.std(100*np.asarray(GB_10p_A_metrics), axis = 0), 3)
std

array([0.44 , 0.499, 0.412, 0.363])

In [36]:
['{}+/-{}'.format(a,s) for a, s in zip(avg, std)]

['97.71+/-0.44', '98.78+/-0.499', '97.45+/-0.412', '98.11+/-0.363']

In [37]:
df = pd.DataFrame(columns = ['Accuracy', 'Precision', 'Recall', 'F1 score'])

metrics = [GB_10p_A_metrics, GB_10p_B_metrics, GB_10p_AB_metrics]
model_names = ['GB_1Op_A', 'GB_1Op_B', 'GB_1Op_AB']

for met, model_name in zip(metrics, model_names):
    
    avg = np.round(np.mean(100*np.asarray(met), axis = 0), 2)
    std = np.round(np.std(100*np.asarray(met), axis = 0), 3)
    df.loc[model_name] = ['{}+/-{}'.format(a,s) for a, s in zip(avg, std)]
    
df

,Accuracy,Precision,Recall,F1 score
GB_1Op_A,97.71+/-0.44,98.78+/-0.499,97.45+/-0.412,98.11+/-0.363
GB_1Op_B,98.79+/-0.331,98.9+/-0.552,99.13+/-0.387,99.01+/-0.271
GB_1Op_AB,99.02+/-0.398,99.25+/-0.58,99.15+/-0.36,99.2+/-0.324


In [38]:
df.to_csv('./results/GB_10p_cls_performance.csv',encoding='utf-8-sig', index = True)  

# 2. Fault classification

In [18]:
class_name = ['Heavy Oil', 'Die Punch', 'Scrapped Wire', 'Die Chip', 'Die Internal', 'PIN']

In [19]:
B6_10p_A_metrics = []
B6_10p_B_metrics = []
B6_10p_AB_metrics = []

B6_10p_A_cms = []
B6_10p_B_cms = []
B6_10p_AB_cms = []

for i, b1_idx, b2_idx, b3_idx, b4_idx, b5_idx, b6_idx in zip(range(5), 
                                                             B1_test_idx, B2_test_idx, B3_test_idx, 
                                                             B4_test_idx, B5_test_idx, B6_test_idx):
    
    B6_Xs_cv = [B6_Xs[0][b1_idx], B6_Xs[1][b2_idx], B6_Xs[2][b3_idx], 
                B6_Xs[3][b4_idx], B6_Xs[4][b5_idx], B6_Xs[5][b6_idx]]
    B6_Ys_cv = [B6_Ys[0][b1_idx], B6_Ys[1][b2_idx], B6_Ys[2][b3_idx], 
                B6_Ys[3][b4_idx], B6_Ys[4][b5_idx], B6_Ys[5][b6_idx]]

    B6_test_X, B6_test_Y = np.concatenate(B6_Xs_cv), np.concatenate(B6_Ys_cv)

    B6_10p_A_model = SSM_Model(np.expand_dims(B6_test_X[:,0,:,:], axis = 1), B6_test_Y, n_ch = 1, n_cls = 6, class_name = class_name, 
                                model_dir = model_dir, model_name = '{:02d}_B6_10p_A'.format(i+1), batch_size = 100, GPU_idx = 6)
    
    B6_10p_A_model.Evaluation()
    B6_10p_A_metrics.append(B6_10p_A_model.cls_metrics)
    B6_10p_A_cms.append(B6_10p_A_model.cm)
    
    B6_10p_B_model = SSM_Model(np.expand_dims(B6_test_X[:,1,:,:], axis = 1), B6_test_Y, n_ch = 1, n_cls = 6, class_name = class_name, 
                                model_dir = model_dir, model_name = '{:02d}_B6_10p_B'.format(i+1), batch_size = 100, GPU_idx = 6)

    B6_10p_B_model.Evaluation()
    B6_10p_B_metrics.append(B6_10p_B_model.cls_metrics)
    B6_10p_B_cms.append(B6_10p_B_model.cm)
    
    B6_10p_AB_model = SSM_Model(B6_test_X, B6_test_Y, n_ch = 2, n_cls = 6, class_name = class_name, 
                                model_dir = model_dir, model_name = '{:02d}_B6_10p_AB'.format(i+1), batch_size = 100, GPU_idx = 6)
    B6_10p_AB_model.Evaluation()
    B6_10p_AB_metrics.append(B6_10p_AB_model.cls_metrics)
    B6_10p_AB_cms.append(B6_10p_AB_model.cm)

model: 01_B6_10p_A_t_accr_0.9984_t_loss_0.006313_v_accr_0.8626_v_loss_1.685970.pt
epoch: 227, lr (starting at 0.01): 0.01
model: 01_B6_10p_B_t_accr_0.9967_t_loss_0.008810_v_accr_0.9094_v_loss_0.731960.pt
epoch: 108, lr (starting at 0.01): 0.01
model: 01_B6_10p_AB_t_accr_0.9997_t_loss_0.002244_v_accr_0.9474_v_loss_0.571496.pt
epoch: 257, lr (starting at 0.01): 0.01
model: 02_B6_10p_A_t_accr_0.9997_t_loss_0.001948_v_accr_0.8713_v_loss_1.585220.pt
epoch: 350, lr (starting at 0.01): 0.01
model: 02_B6_10p_B_t_accr_0.9990_t_loss_0.006569_v_accr_0.9503_v_loss_0.275964.pt
epoch: 120, lr (starting at 0.01): 0.001
model: 02_B6_10p_AB_t_accr_0.9990_t_loss_0.001914_v_accr_0.9327_v_loss_0.561138.pt
epoch: 215, lr (starting at 0.01): 0.001
model: 03_B6_10p_A_t_accr_0.9961_t_loss_0.012610_v_accr_0.8626_v_loss_1.561536.pt
epoch: 297, lr (starting at 0.01): 0.01
model: 03_B6_10p_B_t_accr_1.0000_t_loss_0.001344_v_accr_0.8918_v_loss_0.475617.pt
epoch: 115, lr (starting at 0.01): 1.0000000000000002e-07
mo

In [20]:
np.asarray(B6_10p_AB_cms)

array([[[ 51,   1,   0,   0,   0,   0],
        [  3, 150,   3,   0,   5,   1],
        [  0,   1,  82,   0,   9,   0],
        [  0,   2,   4, 234,   1,   0],
        [  0,   7,   5,   0, 135,   0],
        [  0,   6,   0,   0,   4, 147]],

       [[ 48,   4,   0,   0,   0,   0],
        [ 13, 139,   0,   1,   3,   5],
        [  0,   0,  88,   1,   3,   0],
        [  0,   0,   3, 237,   1,   0],
        [  0,   8,  12,   0, 125,   2],
        [  0,   2,   0,   0,   1, 154]],

       [[ 51,   1,   0,   0,   0,   0],
        [  5, 140,   2,   1,  12,   1],
        [  0,   1,  80,   5,   5,   0],
        [  0,   0,   9, 231,   0,   0],
        [  0,  10,   5,   3, 129,   0],
        [  0,  13,   0,   0,   3, 141]],

       [[ 47,   4,   0,   0,   0,   0],
        [  3, 150,   0,   1,   3,   4],
        [  0,   1,  84,   4,   2,   0],
        [  0,   0,   5, 235,   0,   0],
        [  0,  11,   6,   1, 129,   0],
        [  0,   4,   2,   0,   1, 149]],

       [[ 46,   5,   0,   0,   0

In [24]:
np.save('./results/B6_10p_A_cms', np.asarray(B6_10p_A_cms))
np.save('./results/B6_10p_B_cms', np.asarray(B6_10p_B_cms))
np.save('./results/B6_10p_AB_cms', np.asarray(B6_10p_AB_cms))

In [21]:
avg = np.round(np.mean(100*np.asarray(B6_10p_AB_metrics), axis = 0), 2)
avg

array([[92.66, 89.33, 94.17, 91.61],
       [92.66, 89.01, 88.71, 88.8 ],
       [92.66, 85.29, 90.14, 87.63],
       [92.66, 98.25, 97.59, 97.91],
       [92.66, 90.5 , 87.74, 89.03],
       [92.66, 96.2 , 94.77, 95.38]])

In [54]:
std = np.round(np.std(100*np.asarray(B6_10p_AB_metrics), axis = 0), 2)
std

array([[ 3.33,  3.95, 12.25,  6.85],
       [ 3.33,  4.55,  8.31,  6.29],
       [ 3.33,  5.33,  5.5 ,  3.46],
       [ 3.33,  0.79,  1.52,  0.87],
       [ 3.33,  7.03,  3.11,  3.61],
       [ 3.33,  6.01,  3.51,  4.22]])

In [56]:
['{}+/-{}'.format(a,s) for a, s in zip(avg[0], std[0])]

['90.89+/-3.33', '89.89+/-3.95', '90.61+/-12.25', '89.73+/-6.85']

In [22]:
model_names = ['B6_1Op_A', 'B6_1Op_B', 'B6_1Op_AB']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 score']
tuples = [i for i in itertools.product(model_names, metrics)]
col = pd.MultiIndex.from_tuples(tuples)
df = pd.DataFrame(columns = col)

metrics = [B6_10p_A_metrics, B6_10p_B_metrics, B6_10p_AB_metrics]
class_names = ['Heavy Oil', 'Die Punch', 'Scrapped Wire', 'Die Chip', 'Die Internal', 'PIN']
for c_n, met in zip(np.arange(0, 16, 4), metrics):
    avg = np.round(np.mean(100*np.asarray(met), axis = 0), 2)
    std = np.round(np.std(100*np.asarray(met), axis = 0), 3)
    for i, cls_name in zip(range(len(class_names)), class_names):     
        df.loc[cls_name, c_n:c_n+4] = ['{}+/-{}'.format(a,s) for a, s in zip(avg[i], std[i])]
        
df

B6_1Op_A                                               \
                    Accuracy      Precision         Recall       F1 score   
Heavy Oil      85.31+/-1.395  82.73+/-4.815  82.17+/-5.401  82.31+/-3.794   
Die Punch      85.31+/-1.395  76.32+/-4.596  81.39+/-4.566  78.62+/-3.085   
Scrapped Wire  85.31+/-1.395  82.56+/-5.981  66.93+/-5.208    73.52+/-1.2   
Die Chip       85.31+/-1.395  96.72+/-0.612  95.09+/-1.947  95.88+/-0.764   
Die Internal   85.31+/-1.395  77.59+/-4.395  82.71+/-7.656  79.69+/-2.794   
PIN            85.31+/-1.395  89.53+/-2.817  88.51+/-2.383   88.97+/-1.65   

                    B6_1Op_B                                               \
                    Accuracy      Precision         Recall       F1 score   
Heavy Oil      91.35+/-1.949  84.79+/-7.414  89.16+/-3.327  86.64+/-3.358   
Die Punch      91.35+/-1.949    86.7+/-3.22  83.62+/-5.815  84.97+/-3.212   
Scrapped Wire  91.35+/-1.949  90.28+/-2.598  88.83+/-4.089  89.49+/-2.586   
Die Chip       91.35+/-1.949   98.5+/-0.654    98.0+/-0.81   98.25+/-0.17   
Die Internal   91.35+/-1.949   87.7+/-5.899  88.97+/-2.155  88.23+/-3.584   
PIN            91.35+/-1.949  92.51+/-4.533  93.49+/-2.932  92.94+/-3.059   

                   B6_1Op_AB                                               
                    Accuracy      Precision         Recall       F1 score  
Heavy Oil      92.66+/-1.192  89.33+/-5.743  94.17+/-3.281  91.61+/-4.025  
Die Punch      92.66+/-1.192  89.01+/-2.328   88.71+/-3.51   88.8+/-1.934  
Scrapped Wire  92.66+/-1.192  85.29+/-1.507  90.14+/-3.484  87.63+/-2.126  
Die Chip       92.66+/-1.192    98.25+/-1.3  97.59+/-0.807  97.91+/-0.901  
Die Internal   92.66+/-1.192   90.5+/-3.585   87.74+/-2.29  89.03+/-1.572  
PIN            92.66+/-1.192   96.2+/-3.687  94.77+/-2.883  95.38+/-1.515

In [23]:
df.to_csv('./results/B6_10p_cls_performance.csv',encoding='utf-8-sig', index = True)  